### Exp: 1 working with Non-snapshot

Assumption: every same vehicle in the same parking timespan have a customer that belongs to that car

Vehicle : check , same type, same cam , timestamp in the lifespan +- grace windows (padding periods) x,y is near each other ~~appear at least 2 timeframe~~ on the same x,y on non-snapshot cam

OUTPUT : vehicle_index : {timestamp range, x, y, vehicle type, }

camera used : mbk-14-12-vehicle 

In [1]:
import sys

sys.path.append("/home/jeans/internship/parking-customer-count")

from countpassenger.Config import conf
from countpassenger import Preprocess
from countpassenger.Approch1 import match_cross_to_vehicle, match_reverse_to_vehicle

import countpassenger
import pandas as pd
import numpy as np

import os.path as osp
import os

CURRENT_CAM = "mbk-14-14"

df_vehicle = pd.read_csv(
    osp.join(conf.RESOURCES_RAW_DIR, "2024-04-28/mbk-tourist-vehicle-object-20240428.csv")
)
df_cross = pd.read_csv(
    osp.join(conf.RESOURCES_RAW_DIR, "2024-04-28/mbk-tourist-raw-cross-object-20240428.csv")
)
df_reverse = pd.read_csv(
    osp.join(conf.RESOURCES_RAW_DIR, "2024-04-28/mbk-tourist-raw-reverse-object-20240428.csv")
)

# print(df_vehicle[[*conf.VEHICLE_INTEREST_NON_SNAPSHOT]].to_string())
df_vehicle = Preprocess.df_clean_vehicle(
    df_vehicle=df_vehicle,
)
df_vehicle = Preprocess.filter_camera(df_vehicle, camera_name=CURRENT_CAM + "-vehicle")
df_vehicle = Preprocess.sort_df(df_vehicle, conf.VEHICLE_INTEREST_NON_SNAPSHOT)

# df_vehicle[conf.VEHICLE_INTEREST_NON_SNAPSHOT]

# drop the one that lifetime is less than 4 seconds
# df_vehicle = df_vehicle.loc[(df_vehicle["lifetime"] > 4),]

# create a timestamp_unix at the end of lifespan
# df_vehicle["timestamp_unix_end"] = df_vehicle["timestamp_unix"] + (df_vehicle["lifetime"].astype(np.int64))
# print(df_vehicle[[*conf.VEHICLE_INTEREST_NON_SNAPSHOT,'timestamp_unix','timestamp_unix_end']].to_string())

## Cross vs Vehicle

In [2]:
df_cross = Preprocess.df_clean_customer(df_cross)
df_cross = Preprocess.filter_camera(df_cross, camera_name=CURRENT_CAM)
df_cross = Preprocess.sort_df(df_cross, conf.CUSTOMER_INTEREST_SNAPSHOT)

print(df_cross[[*conf.CUSTOMER_INTEREST_SNAPSHOT, "timestamp_unix"]].to_string())

         camera       timestamp_precise    xmid    ymid  timestamp_unix
2725  mbk-14-14 2024-04-28 02:07:59.195  1989.0   197.5      1714270079
2726  mbk-14-14 2024-04-28 02:08:02.354  1967.5   203.5      1714270082
2719  mbk-14-14 2024-04-28 02:11:47.735   592.0   958.5      1714270307
2720  mbk-14-14 2024-04-28 02:11:47.735  2313.0   158.5      1714270307
2721  mbk-14-14 2024-04-28 02:14:12.454   672.5   949.5      1714270452
2718  mbk-14-14 2024-04-28 02:19:28.528   556.5   907.0      1714270768
2715  mbk-14-14 2024-04-28 02:20:15.761  2198.5   118.0      1714270815
2714  mbk-14-14 2024-04-28 02:20:54.111   734.5   928.0      1714270854
2695  mbk-14-14 2024-04-28 02:31:27.220  1592.0   411.5      1714271487
2696  mbk-14-14 2024-04-28 02:34:19.572  1067.0   731.5      1714271659
2691  mbk-14-14 2024-04-28 02:35:31.236   386.0   930.5      1714271731
2690  mbk-14-14 2024-04-28 02:38:13.951  2191.5   118.0      1714271893
2684  mbk-14-14 2024-04-28 02:39:53.563  1119.0   752.5      171

## Reverse vs Vehicle

In [3]:
df_reverse = Preprocess.df_clean_customer(df_reverse)
df_reverse = Preprocess.filter_camera(df_reverse, camera_name=CURRENT_CAM)
df_reverse = Preprocess.sort_df(df_reverse, conf.CUSTOMER_INTEREST_SNAPSHOT)

# print(df_reverse[[*conf.CUSTOMER_INTEREST_SNAPSHOT, "timestamp_unix"]].to_string())

## TOGETHER


In [4]:
df_vehicle["lifetime"]

2328     1.760
2319     2.999
2320     1.640
2321     4.479
2189    97.092
         ...  
64       1.759
61       1.760
59       1.279
31       4.119
24       2.120
Name: lifetime, Length: 146, dtype: float64

In [5]:
ken = match_reverse_to_vehicle(df_reverse, df_vehicle)
johnny = match_cross_to_vehicle(
    df_cross,
    ken,
)


# df_vehicle = df_vehicle[
#     (df_vehicle["timestamp_unix"] >= 1714284000) & (df_vehicle["timestamp_unix_end"] <= 1714288020)
# ]
# df_vehicle = df_vehicle[
#     [
#         *conf.VEHICLE_INTEREST_NON_SNAPSHOT,
#         "cross_count",
#         "reverse_count",
#         "original_video_name",
#         # "frame_no",
#     ]
# ]

# df_vehicle = Preprocess.sort_df(df_vehicle, conf.VEHICLE_INTEREST_NON_SNAPSHOT)
# print(
#     df_vehicle[
#         [
#             "camera",
#             "timestamp_precise",
#             "lifetime",
#             "cross_count",
#             "reverse_count",
#             "vehicle_type",
#             "plate_number",
#             "timestamp_unix",
#             "timestamp_unix_end",
#             "xmid",
#             "ymid",
#         ]
#     ].to_string()
# )

                 camera       timestamp_precise  lifetime  cross_count  reverse_count vehicle_type plate_number  timestamp_unix  timestamp_unix_end    xmid   ymid
1822  mbk-14-14-vehicle 2024-04-28 06:17:59.954     4.479            0              0          NaN          NaN      1714285079          1714285083  1599.5  169.5
1803  mbk-14-14-vehicle 2024-04-28 06:19:11.174    77.823            1              0          van       338419      1714285151          1714285228  1702.5  190.5
1804  mbk-14-14-vehicle 2024-04-28 06:26:39.256     6.878            1              0       pickup      1นข9532      1714285599          1714285605  1747.0  203.5
1805  mbk-14-14-vehicle 2024-04-28 06:29:48.803     5.279            0              0        sedan       337348      1714285788          1714285793  1462.5  146.0
1781  mbk-14-14-vehicle 2024-04-28 06:32:22.486    95.613            5              1        other       344992      1714285942          1714286037  1656.0  182.5
1759  mbk-14-14-vehicl

In [ ]:
johnny.to_csv(path_or_buf=SAVE_DIR, index = False)

## TODO 
- check cross walk angle?
- allow customer able to be classified to the next car (not found in the current time stamp)?
- check displacement?